## Paqueterías, lectura de las imágenes y coordenadas 

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageChops
from skimage import exposure

img_pro = cv2.imread('imagen_proyectada_sin_marco.png')
img_ort = cv2.imread('imagen_ortogonal_sin_marco.png')

coordenadas_pro = np.array(pd.read_csv('imagen_proyectada_sin_marco.csv', usecols=['X', 'Y']))
coordenadas_ort = np.array(pd.read_csv('imagen_ortogonal_sin_marco.csv', usecols=['X', 'Y']))



## Visualización de las coordenadas en las imágenes

In [4]:
for coord in coordenadas_pro:
    cv2.circle(img_pro, (coord[0], coord[1]), 5, (0, 0, 255), -1) 

# Mostrar la imagen
cv2.imshow('Imagen con coordenadas resaltadas', img_pro)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('img_pro_coordenadas.jpg', img_pro)

True

In [5]:
for coord in coordenadas_ort:
    cv2.circle(img_ort, (coord[0], coord[1]), 5, (0, 0, 255), -1) 

cv2.imshow('Imagen con coordenadas resaltadas', img_ort)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('img_pro_ort.jpg', img_ort)

True

In [ ]:
# Crear una imagen combinada para mostrar ambas imágenes lado a lado
altura_max = max(img_pro.shape[0], img_ort.shape[0])
ancho_total = img_pro.shape[1] + img_ort.shape[1]
imagen_combinada = np.zeros((altura_max, ancho_total, 3), dtype=np.uint8)

# Colocar las imágenes en la imagen combinada
imagen_combinada[:img_pro.shape[0], :img_pro.shape[1]] = img_pro
imagen_combinada[:img_ort.shape[0], img_pro.shape[1]:] = img_ort

# Ajustar las coordenadas de la segunda imagen
coordenadas_ort_ajustadas = coordenadas_ort.copy()
coordenadas_ort_ajustadas[:, 0] += img_pro.shape[1]

# Dibujar líneas entre las coordenadas correspondientes
for coord_pro, coord_ort in zip(coordenadas_pro, coordenadas_ort_ajustadas):
    cv2.line(imagen_combinada, tuple(coord_pro), tuple(coord_ort), (0, 255, 0), 2)  # Línea verde

# Mostrar la imagen combinada
cv2.imshow('Imagen con líneas trazadas', imagen_combinada)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('img_correspondencia.jpg', imagen_combinada)

## Cálculo de la matriz H

In [6]:
def create_homography_matrix(src, dst):
    # Construir la matriz A para Ax = 0
    A = []
    for i in range(len(src)):
        x, y = src[i][0], src[i][1]
        u, v = dst[i][0], dst[i][1]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    
    A = np.array(A)
    
    # Resolver utilizando SVD
    U, S, Vh = np.linalg.svd(A)
    
    # La última fila de Vh corresponde al vector de solución (h11, h12, ..., h33)
    H = Vh[-1].reshape(3, 3)
    
    # Normalizar para que H33 sea igual a 1
    H = H / H[-1, -1]
    
    return H


In [14]:
def mult_matrices(A, *matrices):
    result = np.array(A)
    for matrix in matrices:
        # Multiplicar usando np.dot
        result = np.dot(result, matrix.data)
    return result

def minimos_cuadrados(A, b):
    A = np.array(A)
    b = np.array(b)
    At = A.transpose()
    AtA = mult_matrices(At, A)
    AtAinv = np.linalg.inv(AtA)
    return mult_matrices(AtAinv, At,b)

def t_m(proy, ort, num_imgs):
    A = []
    b = []

    for i in range(num_imgs):
        x, y = proy[i]
        u, v = ort[i]

        A.append([x, y, 1, 0, 0, 0, -u * x, -u * y])
        A.append([0, 0, 0, x, y, 1, -v * x, -v * y])

        b.append([u])
        b.append([v])

    A = np.array(A)
    b = np.array(b)

    #U, S, Vh = np.linalg.svd(A)
    #t_matrix = Vh[-1].reshape(3, 3)
    #t_matrix = t_matrix / t_matrix[-1, -1]

    t_matrix = minimos_cuadrados(A, b)
    t_matrix = np.append(t_matrix,[1])

    return t_matrix

In [18]:
L =t_m(coordenadas_pro, coordenadas_ort,4)
t_r = L.reshape(3, 3)
print(t_r)

[[ 1.37284092e+00 -2.30816363e+00  5.42682656e+02]
 [ 1.98610028e+00  3.27685751e+00 -7.37491228e+02]
 [-1.53156943e-04  4.25123015e-03  1.00000000e+00]]


In [7]:
H = create_homography_matrix(coordenadas_pro, coordenadas_ort)

print("Matriz de homografía H:")
print(H)

Matriz de homografía H:
[[ 1.37284092e+00 -2.30816363e+00  5.42682656e+02]
 [ 1.98610028e+00  3.27685751e+00 -7.37491228e+02]
 [-1.53156943e-04  4.25123015e-03  1.00000000e+00]]


In [8]:
J, _ = cv2.findHomography(coordenadas_pro, coordenadas_ort)

print("Matriz de transformación H:")
print(J)

Matriz de transformación H:
[[ 1.37284092e+00 -2.30816363e+00  5.42682656e+02]
 [ 1.98610028e+00  3.27685751e+00 -7.37491228e+02]
 [-1.53156943e-04  4.25123015e-03  1.00000000e+00]]


## Transformación lineal en la imagen proyectiva.

In [10]:
img_pro = cv2.imread('imagen_proyectada_sin_marco.png')

# Aplicar la transformación utilizando la matriz H
height, width = img_pro.shape[:2]
img_corregida = cv2.warpPerspective(img_pro, H, (width, height))

# Mostrar la imagen corregida
cv2.imshow('Imagen Corregida', img_corregida)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Guardar la imagen corregida
cv2.imwrite('imagen_corregida.jpg', img_corregida)


True

## Diferencias entre la imagene proyectiva corregida y la imagen ortogonal.

In [8]:

img_original_ortogonal = cv2.imread("imagen_ortogonal_sin_marco.png")
img_corregida_ortogonal = cv2.imread("imagen_corregida.jpg")

union = cv2.addWeighted(img_original_ortogonal, 0.5, img_corregida_ortogonal, 0.5, 0)

# Color
union2 = np.zeros((img_original_ortogonal.shape[0], img_original_ortogonal.shape[1], 3), dtype=np.uint8)
union2[:,:,2] = img_original_ortogonal[:,:,0]  # Red channel
union2[:,:,1] = img_corregida_ortogonal[:,:,0]  # Cyan (Green + Blue)
union2[:,:,0] = img_corregida_ortogonal[:,:,0]  # Cyan (Green + Blue)

union3 = np.zeros((img_original_ortogonal.shape[0], img_original_ortogonal.shape[1], 3), dtype=np.uint8)
union3[:,:,1] = img_original_ortogonal[:,:,0]  # Green channel
union3[:,:,2] = img_corregida_ortogonal[:,:,0]  # Magenta (Red + Blue)
union3[:,:,0] = img_corregida_ortogonal[:,:,0]  # Magenta (Red + Blue)

# Montage
union4 = np.hstack((img_original_ortogonal, img_corregida_ortogonal))

# Difference
union5 = cv2.absdiff(img_original_ortogonal, img_corregida_ortogonal)

# Save images
cv2.imwrite("blend_scaling_joint.jpg", union)
#cv2.imwrite("falsecolor_red_cyan.jpg", union2)
cv2.imwrite("falsecolor_green_magenta.jpg", union3)
#cv2.imwrite("montage.jpg", union4)
#cv2.imwrite("difference.jpg", union5)

# Display image
cv2.imshow("Blended Image", union)
cv2.waitKey(0)
cv2.destroyAllWindows()